# featureCounts Compare Exon-Intron Boundaries for Madrasin: DMS vs non-DMS

DataSet: Madrasin HiSeq Data from Oct 2018

## Purpose

This were read counts generated by the program featureCounts After Alignment using Bowtie2 ShapeMapper2 Parms

Aim of this script is to get the ratio of Madrasin vs Controls in exon-intron boundaries for samples that were not treated with DMS 


In [1]:
# Import packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np

In [2]:
# Set variable to find highly expressed genes or transcripts
feature="Gene"

In [3]:
# Set variable to remove non-coding genes
removeNC=False
removeNC_string=""
if removeNC:
    removeNC_string="Coding"
    geneNamesAndTranscriptIDs = pd.read_csv("../data/GeneNameTranscriptID_NCBI_RefSeq_hg38_FromGFFfile.tsv",header=None,sep="\t")
    noncodinggeneNames= list(geneNamesAndTranscriptIDs[geneNamesAndTranscriptIDs[0].str.contains("NR_")|geneNamesAndTranscriptIDs[0].str.contains("XR_")][1].values)
    noncodingtranscriptNames= list(geneNamesAndTranscriptIDs[geneNamesAndTranscriptIDs[0].str.contains("NR_")|geneNamesAndTranscriptIDs[0].str.contains("XR_")][0].values)
    noncodingFeatures = noncodinggeneNames+noncodingtranscriptNames

In [4]:
# Set variable to only look at features that have introns in them
featuresWithIntrons=True
featuresWithIntrons_string=""
if featuresWithIntrons:
    featuresWithIntrons_string="WithIntronicRegions"
    # Get the transcript and genes that have introns within them
    geneNamesAndTranscriptIDs_withIntrons = pd.read_csv("../data/GeneNameTranscriptID_NCBI_RefSeq_hg38_FromGFFfile_OnlyOnesWithIntrons.tsv",header=None,sep="\t")
    print geneNamesAndTranscriptIDs_withIntrons.shape
    geneNamesAndTranscriptIDs_withIntrons.head()

(153118, 2)


In [5]:
# Set variable to determine how many expressed features to look at
# If set to 0, then get all features
numFeatures=50

In [6]:
# which end
whichend=str(5)

In [7]:
# Get highly expressed features from each file for the transcriptome data
top_features_Transcriptome = []
for filename in os.listdir("../tmp/Madrasin_HiSeq_October2018_alignments/"):
    if ("_Intron"+whichend+"p_featureCounts_JustCounts.txt" in filename):
        print filename
        # Read the salmon output file
        with open("../tmp/Madrasin_HiSeq_October2018_alignments/"+filename) as f:
            lines = f.readlines()
        with open("../tmp/Madrasin_HiSeq_October2018_alignments/"+filename.split('.')[0]+"_DropHeader.txt",'w') as fw:
            fw.writelines(lines[1:])
        data_to_read = pd.read_csv("../tmp/Madrasin_HiSeq_October2018_alignments/"+filename.split('.')[0]+"_DropHeader.txt",header=0,sep="\t")
        data_to_read.columns = ["Name","Length","Count"]
        # If features with Introns flag is set, only get features that have introns in them
        if featuresWithIntrons:
            # Only features with introns
            data_to_read = data_to_read[(data_to_read["Name"].isin(geneNamesAndTranscriptIDs_withIntrons[0])|data_to_read["Name"].isin(geneNamesAndTranscriptIDs_withIntrons[1]))]
        # If removeNC flag is set, only get coding features
        if removeNC:
            data_to_read = data_to_read[~(data_to_read["Name"].str.contains("NR_")|data_to_read["Name"].str.contains("XR_")|data_to_read["Name"].isin(noncodingFeatures))]
        # Calculate TPMs
        RPKs = 1000*(data_to_read["Count"]/data_to_read["Length"])
        total_RPKs = RPKs.sum()
        TPMs = 1000000*(RPKs/total_RPKs)
        data_to_read = data_to_read.assign(TPM=TPMs)
        # Sort the data by TPMs
        data_to_read_sorted = data_to_read.sort_values(by=["TPM"],ascending=False)
        # Grab the names of the first numFeatures features
        if numFeatures==0:
            top_features_Transcriptome.extend(list(data_to_read_sorted["Name"].values))
        else:
            top_features_Transcriptome.extend(list(data_to_read_sorted.iloc[range(numFeatures),0].values))

NoDMSSample_CTCTCTAC_S2_ExonIntron5p_featureCounts_JustCounts.txt
DMSTreatedSample_TAGGCATG_S1_ExonIntron5p_featureCounts_JustCounts.txt


In [8]:
# Combine the two feature files together to get a unique list
unique_top_features = list(set(top_features_Transcriptome))
print(len(unique_top_features))

72


In [9]:
# Create a data frame with unique top features as column
feature_readCounts_transcriptome = pd.DataFrame({"feature":sorted(unique_top_features)})
# For the unique features, get the TPM for each of them and store in a data frame
# Read in the Count files 
for filename in os.listdir("../tmp/Madrasin_HiSeq_October2018_alignments/"):
    current_columns = list(feature_readCounts_transcriptome.columns.values)
    if ("ExonIntron"+whichend+"p_featureCounts_JustCounts_DropHeader.txt" in filename):
        print filename
        current_columns.append(filename.split("_ExonIntron")[0])
        data_to_read = pd.read_csv("../tmp/Madrasin_HiSeq_October2018_alignments/"+filename,header=0,sep="\t")
        data_to_read.columns = ["Name","Length","Count"]
        # Calculate TPMs
        RPKs = 1000*(data_to_read["Count"]/data_to_read["Length"])
        total_RPKs = RPKs.sum()
        TPMs = 1000000*(RPKs/total_RPKs)
        data_to_read = data_to_read.assign(TPM=TPMs)
        data_GetFeatures = data_to_read[data_to_read["Name"].isin(unique_top_features)]
        data_sortedBy_FeatureName = data_GetFeatures.sort_values(by="Name")
        data_sortedBy_FeatureName = data_sortedBy_FeatureName.reset_index(drop=True)
        feature_readCounts_transcriptome = pd.concat([feature_readCounts_transcriptome,data_sortedBy_FeatureName["TPM"]],axis=1,ignore_index=True)
        feature_readCounts_transcriptome.columns = current_columns

DMSTreatedSample_TAGGCATG_S1_ExonIntron5p_featureCounts_JustCounts_DropHeader.txt
NoDMSSample_CTCTCTAC_S2_ExonIntron5p_featureCounts_JustCounts_DropHeader.txt


In [10]:
feature_readCounts_transcriptome = feature_readCounts_transcriptome.iloc[:,range(1,feature_readCounts_transcriptome.shape[1])]
feature_readCounts_transcriptome.index = sorted(unique_top_features)
print feature_readCounts_transcriptome.shape
feature_readCounts_transcriptome.head()

(72, 2)


,DMSTreatedSample_TAGGCATG_S1,NoDMSSample_CTCTCTAC_S2
ACTB,8648.644113,7566.765250
ACTG1,2431.150595,1786.700069
AEN,837.012411,531.387287
ATF4,819.664307,759.797528
ATP5F1B,802.709685,2295.300607


In [11]:
feature_readCounts_transcriptome

,DMSTreatedSample_TAGGCATG_S1,NoDMSSample_CTCTCTAC_S2
ACTB,8648.644113,7566.765250
ACTG1,2431.150595,1786.700069
AEN,837.012411,531.387287
ATF4,819.664307,759.797528
ATP5F1B,802.709685,2295.300607
CALR,1294.579868,3068.165441
CCND2,828.943525,968.306271
CD48,671.196810,1546.487191
CD70,1489.142742,882.516570
CD74,3907.453162,3916.404007
